# 52 文本预处理

### 文本的常见预处理步骤。这些步骤通常包括:
#### 1.将文本作为字符串加载到内存中。
#### 2.将字符串拆分为词元（如单词和字符)。
#### 3.建立一个词表，将拆分的词元映射到数字索引。
#### 4.将文本转换为数字索引序列，方便模型操作。

In [2]:
import collections
import re
from d2l import torch as d2l

## 1 读取数据集

下面的函数将数据集读取到由多条文本行组成的列表中，其中每条文本行都是一个字符串。 为简单起见，我们在这里忽略了标点符号和字母大写。

In [3]:
d2l.DATA_HUB['time_machine'] = (d2l.DATA_URL + 'timemachine.txt',
                                '090b5e7e70c295757f55df93cb0a180b9691891a')

def read_time_machine():  #@save
    """将时间机器数据集加载到文本行的列表中"""
    with open(d2l.download('time_machine'), 'r') as f:
        lines = f.readlines()
    # re.sub()，该函数主要用于替换字符串中的匹配项  '[^A-Za-z]+', ' '表示匹配单个非字母，并将单个非字母替换为一个空格
    return [re.sub('[^A-Za-z]+', ' ', line).strip().lower() for line in lines]

lines = read_time_machine()
print(f'# 文本总行数: {len(lines)}')
print(lines[0])
print(lines[10])

# 文本总行数: 3221
the time machine by h g wells
twinkled and his usually pale face was flushed and animated the


## 2 词元化

下面的tokenize函数将文本行列表（lines）作为输入， 列表中的每个元素是一个文本序列（如一条文本行）。 </br>
每个文本序列又被拆分成一个词元列表，词元（token）是文本的基本单位。</br>
最后，返回一个由词元列表组成的列表，其中的每个词元都是一个字符串（string）。

In [4]:
def tokenize(lines, token='word'):  #@save
    """将文本行拆分为单词或字符词元"""
    if token == 'word':
        return [line.split() for line in lines]
    elif token == 'char':
        return [list(line) for line in lines]
    else:
        print('错误：未知词元类型：' + token)

tokens = tokenize(lines)
for i in range(11):
    print(tokens[i])

['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
[]
[]
[]
[]
['i']
[]
[]
['the', 'time', 'traveller', 'for', 'so', 'it', 'will', 'be', 'convenient', 'to', 'speak', 'of', 'him']
['was', 'expounding', 'a', 'recondite', 'matter', 'to', 'us', 'his', 'grey', 'eyes', 'shone', 'and']
['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']


## 3 词表

词元的类型是字符串，而模型需要的输入是数字，因此这种类型不方便模型使用。 </br>
现在，让我们构建一个字典，通常也叫做词表（vocabulary）， 用来将字符串类型的词元映射到从0开始的数字索引中。</br>
我们先将训练集中的所有文档合并在一起，对它们的唯一词元进行统计， 得到的统计结果称之为语料（corpus）。</br>
然后根据每个唯一词元的出现频率，为其分配一个数字索引。 很少出现的词元通常被移除，这可以降低复杂性。 </br>
另外，语料库中不存在或已删除的任何词元都将映射到一个特定的未知词元“\<unk\>”。 </br>
我们可以选择增加一个列表，用于保存那些被保留的词元， 例如：填充词元（“\<pad\>”）； 序列开始词元（“\<bos\>”）； 序列结束词元（“\<eos\>”）。

In [5]:
class Vocab:  #@save
    """文本词表"""
    # reserved_tokens用于告知句子的开始和结束
    def __init__(self, tokens=None, min_freq=0, reserved_tokens=None):
        if tokens is None:
            tokens = []
        if reserved_tokens is None:
            reserved_tokens = []
        # 按出现频率排序
        counter = count_corpus(tokens)
        # key=lambda x: x[1]按计数器的value排序
        self._token_freqs = sorted(counter.items(), key=lambda x: x[1],
                                   reverse=True)
        # 未知词元的索引为0
        self.idx_to_token = ['<unk>'] + reserved_tokens
        self.token_to_idx = {token: idx
                             for idx, token in enumerate(self.idx_to_token)}
        for token, freq in self._token_freqs:
            if freq < min_freq:
                break
            if token not in self.token_to_idx:
                # 数对应词
                self.idx_to_token.append(token)
                # 词对应数
                self.token_to_idx[token] = len(self.idx_to_token) - 1

    def __len__(self):
        return len(self.idx_to_token)

    # 给token返回index
    def __getitem__(self, tokens):
        if not isinstance(tokens, (list, tuple)):
            # 字典中根据key=tokens查找value 没有则返回unk
            return self.token_to_idx.get(tokens, self.unk)
        return [self.__getitem__(token) for token in tokens]

    # 给index返回token
    def to_tokens(self, indices):
        # isinstance对象的类型与参数二的类型（classinfo）相同则返回 True 是list或是tuple
        if not isinstance(indices, (list, tuple)):
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices]

    @property
    def unk(self):  # 未知词元的索引为0
        return 0

    @property
    def token_freqs(self):
        return self._token_freqs

def count_corpus(tokens):  #@save
    """统计词元的频率"""
    # 这里的tokens是1D列表或2D列表
    if len(tokens) == 0 or isinstance(tokens[0], list):
        # 将词元列表展平成一个列表
        tokens = [token for line in tokens for token in line]
    # collections.Counter可以支持方便、快速的计数，将元素数量统计，然后计数并返回一个字典，键为元素，值为元素个数
    return collections.Counter(tokens)

首先使用时光机器数据集作为语料库来构建词表，然后打印前几个高频词元及其索引

In [6]:
vocab = Vocab(tokens)
print(list(vocab.token_to_idx.items())[:10])

[('<unk>', 0), ('the', 1), ('i', 2), ('and', 3), ('of', 4), ('a', 5), ('to', 6), ('was', 7), ('in', 8), ('that', 9)]


将每一条文本行转换成一个数字索引列表

In [7]:
for i in [0, 10]:
    print('文本:', tokens[i])
    print('索引:', vocab[tokens[i]])

文本: ['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
索引: [1, 19, 50, 40, 2183, 2184, 400]
文本: ['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']
索引: [2186, 3, 25, 1044, 362, 113, 7, 1421, 3, 1045, 1]


## 4 整合所有功能

在使用上述函数时，我们将所有功能打包到load_corpus_time_machine函数中，</br>
该函数返回corpus(词元索引列表)和vocab(时光机器语料库的词表)。我们在这里所做的改变是:</br>
1 为了简化后面章节中的训练，我们使用字符(而不是单词)实现文本词元化</br>
2 时光机器数据集中的每个文本行不一定是一个句子或一个段落，还可能是一个单词，因此返回的corpus仅处理为单个列表，而不是使用多词元列表构成的一个列表。


In [8]:
def load_corpus_time_machine(max_tokens=-1):  
    """返回时光机器数据集的词元索引列表和词表"""
    lines = read_time_machine()
    tokens = tokenize(lines, 'char')
    vocab = Vocab(tokens)
    # 因为时光机器数据集中的每个文本行不一定是一个句子或一个段落，
    # 所以将所有文本行展平到一个列表中
    corpus = [vocab[token] for line in tokens for token in line]
    if max_tokens > 0:
        corpus = corpus[:max_tokens]
    return corpus, vocab

corpus, vocab = load_corpus_time_machine()
len(corpus), len(vocab)

(170580, 28)

文本是序列数据的一种最常见的形式之一。
</br>
为了对文本进行预处理，我们通常将文本拆分为词元，构建词表将词元字符串映射为数字索引，并将文本数据转换为词元索引以供模型操作